# Do all analysis and plots

In [ ]:
import os

import numpy as np
import matplotlib.pyplot as pl
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import linregress
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xarray as xr

In [ ]:
os.makedirs('../plots', exist_ok=True)

In [ ]:
pl.style.use('../defaults.mplstyle')

In [ ]:
# no CH4 removal, i.e. fair default runs, with no temperature rebasing
noch4r = xr.open_dataset('../results/noCH4R_cal-1.4.0.nc')

In [ ]:
# CH4 removal runs with temperature rebasing
ch4r = xr.open_dataset('../results/CH4R_cal-1.4.0.nc')

In [ ]:
# CH4 lifetime in the default case
noch4r_lifetime = xr.open_dataset('../results/noCH4R_lifetime_cal-1.4.0.nc')

In [ ]:
config0 = noch4r_lifetime.config[0].values[()]

## These are raw outputs for the first ssp119 ensemble member in 2010 and should not differ.

In [ ]:
noch4r.temperature.sel(timebound=2010, scenario='ssp119', config=config0, layer=0).data[()]

In [ ]:
ch4r.temperature.sel(timebound=2010, scenario='ssp119', config=config0).data[()]

In [ ]:
temp_norm_20032022 = np.ones(21)
temp_norm_20032022[0] = 0.5
temp_norm_20032022[-1] = 0.5

In [ ]:
np.average(noch4r.temperature.sel(timebound=np.arange(2003, 2024), scenario='ssp119', config=config0, layer=0), weights=temp_norm_20032022)

In [ ]:
np.average(ch4r.temperature.sel(timebound=np.arange(2003, 2024), scenario='ssp119', config=config0), weights=temp_norm_20032022)

## ssp119 peak warming

Using the 1.03C present-day warming convention

In [ ]:
(
    noch4r.temperature.loc[
        dict(scenario='ssp119', layer=0)
    ] - np.average(
        noch4r.temperature.loc[
            dict(scenario='ssp119', layer=0, timebound=np.arange(2003,2024))
        ], axis=0, weights=temp_norm_20032022
    ) + 1.03
).median(dim='config').max()

In [ ]:
(
    noch4r.temperature.loc[
        dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', layer=0)
    ] - np.average(
        noch4r.temperature.loc[
            dict(scenario='COFFEE1.1 EN_NPi2020_400f_lowBECCS', layer=0, timebound=np.arange(2003,2024))
        ], axis=0, weights=temp_norm_20032022
    ) + 1.03
).median(dim='config').max()

## Main analysis

In [ ]:
ecs_tcr = pd.read_csv('../results/ecs_tcr_cal-1.4.0.csv', index_col=0)

In [ ]:
colors = {
    'ssp119': '#00a9cf',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': '#707070', #'#B8BDAA',
    'ssp534-over': '#92397a'
}

In [ ]:
scenarios = ['ssp119', 'COFFEE1.1 EN_NPi2020_400f_lowBECCS', 'ssp534-over']
labels = {
    'ssp119': 'SSP1-1.9',
    'COFFEE1.1 EN_NPi2020_400f_lowBECCS': 'IMP-Neg',
    'ssp534-over': 'SSP5-3.4-overshoot'
}

In [ ]:
def powerfit(x, a, b, c):
    return a + b * x**c

def squarerootfit(x, b):
    return 1.5 + b * x**0.5

def squarefit(x, b):
    return b * (x-1.5)**2

In [ ]:
quantiles = [.05, .25, .50, .75, .95]
mod = {}
res_all = {}
x_p = {}
df_p = {}
p = {}
cov = {}
exceeds = {}

modsr = {}
res_allsr = {}
x_psr = {}
df_psr = {}
psr = {}
covsr = {}

modsq = {}
res_allsq = {}
x_psq = {}
df_psq = {}
psq = {}
covsq = {}

regress_result = {}

for iscen, scenario in enumerate(scenarios):
    over1p5 = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )
    ) >= 1.5
    x = (
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        ).data[over1p5]/1000
    )
    y = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data[over1p5]
    )
    exceeds[scenario] = (
        np.max(
            ch4r.temperature.loc[
                dict(scenario=scenario)
            ] - np.average(
                ch4r.temperature.loc[
                    dict(scenario=scenario, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data > 1.5
    )
    xe = (
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        ).data[over1p5 & ~exceeds[scenario]]/1000
    )
    ye = (
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        ).data[over1p5 & ~exceeds[scenario]]
    )
    
    d = {'peak': ye, 'ch4': xe}
    dsr = {'peak': ye, 'ch4': xe}
    df = pd.DataFrame(data=d)
    
    # scipy curve_fit
    p[scenario], cov[scenario] = curve_fit(powerfit, xe, ye, maxfev=5000)
    psr[scenario], covsr[scenario] = curve_fit(squarerootfit, xe, ye)
    psq[scenario], covsq[scenario] = curve_fit(squarefit, ye, xe)
    regress_result[scenario] = linregress(ye, xe)
    # use the best estimate of the exponent from this to feed into the quantile regression
    
    # statsmodels fit
    mod[scenario] = smf.quantreg(f'peak ~  I(ch4 ** {p[scenario][2]})', df)
    modsr[scenario] = smf.quantreg(f'peak ~ I(ch4 ** 0.5)', df)
    modsq[scenario] = smf.quantreg(f'ch4 ~ I(peak ** 2)', df)
    
    # get all result instances in a list
    res_all[scenario] = [mod[scenario].fit(q=q) for q in quantiles]
    x_p[scenario] = np.linspace(0, x.max(), 100)
    df_p[scenario] = pd.DataFrame({'ch4': x_p})

    res_allsr[scenario] = [modsr[scenario].fit(q=q) for q in quantiles]
    x_psr[scenario] = np.linspace(0, x.max(), 100)
    df_psr[scenario] = pd.DataFrame({'ch4': x_psr})

    res_allsq[scenario] = [modsq[scenario].fit(q=q) for q in quantiles]
    x_psq[scenario] = np.linspace(0, y.max(), 100)
    df_psq[scenario] = pd.DataFrame({'peak': x_psq})

In [ ]:
regress_result

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
#     for qm, res in zip(quantiles, res_all[scenario]):
#         q[qm] = res.predict({'peak': x_p[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
#     ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
#     ax[iscen].plot(x_p[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (GtCH$_4$)')
    ax[iscen].set_ylabel('Peak warming (K)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(1, 4.5)
fig.tight_layout()
pl.savefig('../plots/cumCH4_peakT_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_peakT_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
#     for qm, res in zip(quantiles, res_all[scenario]):
#         q[qm] = res.predict({'peak': x_p[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
#     ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
#     ax[iscen].plot(x_p[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_ylabel('Cumulative methane removal (PgCH$_4$)')
    ax[iscen].set_xlabel('Peak warming (K)')
    ax[iscen].set_ylim(-5, 130)
    ax[iscen].set_xlim(1, 4.5)
fig.tight_layout()
pl.savefig('../plots/peakT_cumCH4_cal-1.4.0.png')
pl.savefig('../plots/peakT_cumCH4_cal-1.4.0.pdf')

## gigatons removal required in ssp119 to avoid peak 1.5C

In [ ]:
for scenario in scenarios:
    print(scenario, 
        np.percentile(-np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, (5, 50, 95, 100))
    )

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    
    q = {}
    for qm, res in zip(quantiles, res_all[scenario]):
        q[qm] = res.predict({'ch4': x_p[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
    #ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    #ax[iscen].plot(x_p[scenario], q[.5], color='k')
    ax[iscen].fill_between(x_p[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    ax[iscen].plot(x_p[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    ax[iscen].set_ylabel('Peak warming (°C)')
    ax[iscen].set_xlim(0, 120)
    ax[iscen].set_ylim(1, 4.5)
    

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color="None", label='Limited to 1.5°C', markerfacecolor='k', markersize=np.sqrt(8)),
    Line2D([0], [0], marker='o', color='None', label='Not Limited to 1.5°C', markerfacecolor='w', markeredgecolor='k', markersize=np.sqrt(8), lw=0.4),
    Patch(facecolor='0.7', lw=0, label='Quantile regression 5-95%'),
    Line2D([0], [0], color='k', lw=1.5, label='Quantile regression median'),
]
ax[0].legend(handles=legend_elements, loc='upper left', frameon=False)

fig.tight_layout()
pl.savefig('../plots/cumCH4_peakT_quant_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_peakT_quant_cal-1-4-0.pdf')

In [ ]:
for scenario in scenarios:
    for qidx in [0, 2, 4]:
        print(scenario, quantiles[qidx])
        print(res_all[scenario][qidx].params)
        print()

In [ ]:
for scenario in scenarios:
    for qidx in [0, 2, 4]:
        print(scenario, quantiles[qidx])
        print(res_allsq[scenario][qidx].params)
        print()

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    
    q = {}
    for qm, res in zip(quantiles, res_allsr[scenario]):
        q[qm] = res.predict({'ch4': x_psr[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
    ax[iscen].fill_between(x_psr[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    ax[iscen].plot(x_psr[scenario], q[.5], color='k')

    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    ax[iscen].set_ylabel('Peak warming (°C)')
    ax[iscen].set_xlim(0, 120)
    ax[iscen].set_ylim(1, 4.5)
    

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color="None", label='Limited to 1.5°C', markerfacecolor='k', markersize=np.sqrt(8)),
    Line2D([0], [0], marker='o', color='None', label='Not Limited to 1.5°C', markerfacecolor='w', markeredgecolor='k', markersize=np.sqrt(8), lw=0.4),
    Patch(facecolor='0.7', lw=0, label='Quantile regression 5-95%'),
    Line2D([0], [0], color='k', lw=1.5, label='Quantile regression median'),
]
ax[0].legend(handles=legend_elements, loc='upper left', frameon=False)

fig.tight_layout()
pl.savefig('../plots/cumCH4_peakT_quantsquarerootfit_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_peakT_quantsquarerootfit_cal-1-4-0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    
    q = {}
    for qm, res in zip(quantiles, res_allsq[scenario]):
        q[qm] = res.predict({'peak': x_psq[scenario]})
        # get prediction for the model and plot
        # here we use a dict which works the same way as the df in ols
#    ax[iscen].fill_between(x_psq[scenario], q[.05], q[.95], color='k', alpha=0.3, lw=0)
    ax[iscen].plot(x_psq[scenario], q[.5], color='k')

#    ax[iscen].plot(np.linspace(1.5, 4.5, 301), squarefit(np.linspace(1.5, 4.5, 301), psq[scenario]), color='k')
    
    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    if iscen==0:
        ax[iscen].set_ylabel('Cumulative methane removal (PgCH$_4$)')
    ax[iscen].set_xlabel('Peak warming (°C)')
    ax[iscen].set_ylim(0, 120)
    ax[iscen].set_xlim(1, 4.5)
    

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color="None", label='Limited to 1.5°C', markerfacecolor='k', markersize=np.sqrt(8)),
    Line2D([0], [0], marker='o', color='None', label='Overshoots 1.5°C', markerfacecolor='w', markeredgecolor='k', markersize=np.sqrt(8), lw=0.4),
    # Patch(facecolor='0.7', lw=0, label='Quantile regression 5-95%'),
    Line2D([0], [0], color='k', lw=1.5, label='Eq. (1)'),
]
ax[2].legend(handles=legend_elements, loc='lower right', frameon=False)

fig.tight_layout()
pl.savefig('../plots/peakT_cumCH4_quantsquarefit_cal-1.4.0.png')
pl.savefig('../plots/peakT_cumCH4_quantsquarefit_cal-1-4-0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[~exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].scatter(
        np.max(
            noch4r.temperature.loc[
                dict(scenario=scenario, layer=0)
            ] - np.average(
                noch4r.temperature.loc[
                    dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))
                ], axis=0, weights=temp_norm_20032022
            ) + 1.03, axis=0
        )[exceeds[scenario]],
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    


    ax[iscen].plot(
        np.linspace(1.5, 4.5, 301), 
        regress_result[scenario].slope*np.linspace(1.5, 4.5, 301) + regress_result[scenario].intercept,
        color='k'
    )
    
    ax[iscen].set_title(f'({chr(97+iscen)}) {labels[scenario]}')
    if iscen==0:
        ax[iscen].set_ylabel('Cumulative methane removal (PgCH$_4$)')
    ax[iscen].set_xlabel('Peak warming (°C)')
    ax[iscen].set_ylim(0, 120)
    ax[iscen].set_xlim(1, 4.5)
    

# Custom legend
legend_elements = [
    Line2D([0], [0], marker='o', color="None", label='Limited to 1.5°C', markerfacecolor='k', markersize=np.sqrt(8)),
    Line2D([0], [0], marker='o', color='None', label='Overshoots 1.5°C', markerfacecolor='w', markeredgecolor='k', markersize=np.sqrt(8), lw=0.4),
    # Patch(facecolor='0.7', lw=0, label='Quantile regression 5-95%'),
    Line2D([0], [0], color='k', lw=1.5, label='Eq. (1)'),
]
ax[2].legend(handles=legend_elements, loc='lower right', frameon=False)

fig.tight_layout()
pl.savefig('../plots/peakT_cumCH4_linear_cal-1.4.0.png')
pl.savefig('../plots/peakT_cumCH4_linear_cal-1-4-0.pdf')

In [ ]:
for scenario in scenarios:
    for qidx in [0, 2, 4]:
        print(scenario, quantiles[qidx])
        print(res_allsr[scenario][qidx].params)
        print()

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls = '--'
    );
    ax.plot(
        ch4r.timepoint, 
        np.median(ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
    
hands, labs = ax.get_legend_handles_labels()

hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')

ax.set_xlim(2015, 2150)
ax.set_ylim(-400, 600)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Tg CH$_4$ yr$^{-1}$')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median net methane emissions');
pl.savefig('../plots/netCH4emissions_cal-1.4.0.png')
pl.savefig('../plots/netCH4emissions_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax.set_xlim(2015, 2150)
ax.set_ylim(0, 900)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('Tg CH$_4$ yr$^{-1}$')
ax.legend(frameon=False)
ax.set_title('Ensemble median gross methane removal');
pl.savefig('../plots/grossCH4removal_cal-1.4.0.png')
pl.savefig('../plots/grossCH4removal_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(12/2.54, 6/2.54))
for scenario in scenarios:
    ax[1].plot(
        noch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax[1].plot(
        ch4r.timepoint, 
        np.median(ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax[1].set_xlim(2000, 2300)
ax[1].set_ylim(-400, 600)
ax[1].axhline(0, lw=0.5, ls=':', color='k')
#ax[1].set_ylabel('Mt CH$_4$ yr$^{-1}$')
ax[1].set_title('(b) Net methane emissions');

hands = []
labs = []
#hands, labs = ax[1].get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax[1].legend(hands, labs, frameon=False)

for scenario in scenarios:
    ax[0].plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
ax[0].set_xlim(2000, 2300)
ax[0].set_ylim(0, 900)
ax[0].axhline(0, lw=0.5, ls=':', color='k')
ax[0].set_ylabel('Tg CH$_4$ yr$^{-1}$')
ax[0].legend(frameon=False)
ax[0].set_title('(a) Gross methane removal');
fig.tight_layout()

pl.savefig('../plots/netCH4emissions_grossCH4removal_cal-1.4.0.png')
pl.savefig('../plots/netCH4emissions_grossCH4removal_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(
            (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ), axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(0.7, 2.5)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('°C relative to 1850-1900')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median global mean surface temperature');
pl.savefig('../plots/temperature-cal_1.4.0.png')
pl.savefig('../plots/temperature-cal_1.4.0.pdf')

In [ ]:
np.median(
    (
        noch4r.temperature.loc[dict(scenario='ssp119', layer=0)] -
        np.average(noch4r.temperature.loc[dict(scenario='ssp119', layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
        + 1.03
    ),
    axis=1
).max()

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ) - (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ),
        color=colors[scenario],
        label=labels[scenario]
    );

ax.set_xlim(2000, 2300)
ax.set_ylim(-0.05, 0.8)
ax.set_ylabel('°C')
ax.axhline(1.5, lw=0.5, ls=':', color='k')
ax.legend(frameon=False)
ax.set_title('Ensemble median avoided warming');
pl.savefig('../plots/avoidedwarming_cal-1.4.0.png')
pl.savefig('../plots/avoidedwarming_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(12/2.54, 6/2.54))
for scenario in scenarios:
    # ax[0].fill_between(
    #     ch4r.timebound, 
    #     np.percentile(ch4r.temperature.loc[dict(scenario=scenario)] -
    #             np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03, 5, axis=1), 
    #     np.percentile(ch4r.temperature.loc[dict(scenario=scenario)] -
    #             np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03, 95, axis=1), 
    #     color=colors[scenario],
    #     lw=0.5,
    #     alpha=0.2,
    # );
    ax[0].plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ),
        color=colors[scenario],
        ls='--',
    );
    ax[0].plot(
        ch4r.timebound, 
        np.median(ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03, axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax[0].get_legend_handles_labels()
# hands.append(Line2D([0], [0], color='k', lw=1.5))
# hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
# labs.append('With methane removal')
# labs.append('Unaltered scenario')
ax[0].set_xlim(2000, 2300)
ax[0].set_ylim(0.6, 2.4)
ax[0].axhline(0, lw=0.5, ls=':', color='k')
ax[0].set_ylabel('°C relative to 1850-1900')
ax[0].axhline(1.5, lw=0.5, ls=':', color='k')
# ax[0].legend(hands, labs, frameon=False)
ax[0].set_title('(a) Global mean surface temperature');

for scenario in scenarios:
    # ax[1].fill_between(
    #     ch4r.timebound, 
    #     np.percentile(
    #         (
    #             noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
    #             np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03
    #         ) - (
    #             ch4r.temperature.loc[dict(scenario=scenario)] -
    #             np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03
    #         ),
    #         5,
    #         axis=1
    #     ),
    #     np.percentile(
    #         (
    #             noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
    #             np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03
    #         ) - (
    #             ch4r.temperature.loc[dict(scenario=scenario)] -
    #             np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    #             + 1.03
    #         ),
    #         95,
    #         axis=1
    #     ),
    #     color=colors[scenario],
    #     lw=0.5,
    #     alpha=0.2,
    # );
    ax[1].plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ) - (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ),
        color=colors[scenario],
        label=labels[scenario]
    );

ax[1].set_xlim(2000, 2300)
ax[1].set_ylim(-0.05, 1.0)
ax[1].set_ylabel('°C')
ax[1].axhline(0, lw=0.5, ls=':', color='k')
ax[1].legend(frameon=False)
ax[1].set_title('(b) Avoided warming');

fig.tight_layout()
pl.savefig('../plots/temperature-cal_1.4.0.png')
pl.savefig('../plots/temperature-cal_1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 2, figsize=(12/2.54, 6/2.54))
for scenario in scenarios:
    ax[0].fill_between(
        ch4r.timebound, 
        np.percentile(ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03, 5, axis=1), 
        np.percentile(ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03, 95, axis=1), 
        color=colors[scenario],
        lw=0.5,
        alpha=0.2,
    );
    ax[0].plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ),
        color=colors[scenario],
        ls='--',
    );
    ax[0].plot(
        ch4r.timebound, 
        np.median(ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03, axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax[0].get_legend_handles_labels()
# hands.append(Line2D([0], [0], color='k', lw=1.5))
# hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
# labs.append('With methane removal')
# labs.append('Unaltered scenario')
ax[0].set_xlim(2000, 2300)
ax[0].set_ylim(0.6, 2.4)
ax[0].axhline(0, lw=0.5, ls=':', color='k')
ax[0].set_ylabel('°C relative to 1850-1900')
ax[0].axhline(1.5, lw=0.5, ls=':', color='k')
# ax[0].legend(hands, labs, frameon=False)
ax[0].set_title('(a) Global mean surface temperature');

for scenario in scenarios:
    ax[1].fill_between(
        ch4r.timebound, 
        np.percentile(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ) - (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            5,
            axis=1
        ),
        np.percentile(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ) - (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            95,
            axis=1
        ),
        color=colors[scenario],
        lw=0.5,
        alpha=0.2,
    );
    ax[1].plot(
        noch4r.timebound, 
        np.median(
            (
                noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
                np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ) - (
                ch4r.temperature.loc[dict(scenario=scenario)] -
                np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
                + 1.03
            ),
            axis=1
        ),
        color=colors[scenario],
        label=labels[scenario]
    );

ax[1].set_xlim(2000, 2300)
ax[1].set_ylim(-0.05, 1.3)
ax[1].set_ylabel('°C')
ax[1].axhline(0, lw=0.5, ls=':', color='k')
ax[1].legend(frameon=False)
ax[1].set_title('(b) Avoided warming');

fig.tight_layout()
pl.savefig('../plots/temperature-uncert-cal_1.4.0.png')
pl.savefig('../plots/temperature-uncert-cal_1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.concentration.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--'
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.concentration.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(0, 2300)
ax.set_ylabel('ppb')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane concentrations');
pl.savefig('../plots/CH4concentrations_cal-1.4.0.png')
pl.savefig('../plots/CH4concentrations_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_ch4.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_xlim(2000, 2300)
ax.set_ylim(-1, 0.8)
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.set_ylabel('W m$^{-2}$')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane radiative forcing');
pl.savefig('../plots/CH4forcing_cal-1.4.0.png')
pl.savefig('../plots/CH4forcing_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='Ozone')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_o3.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(-0.1, 0.6)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median ozone radiative forcing');
pl.savefig('../plots/O3forcing_cal-1.4.0.png')
pl.savefig('../plots/O3forcing_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        ls = '--',
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_sum.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(1, 5)
ax.set_ylabel('W m$^{-2}$')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median radiative forcing');
pl.savefig('../plots/totalforcing_cal-1.4.0.png')
pl.savefig('../plots/totalforcing_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1,3,figsize=(18/2.54, 6/2.54))
for scenario in scenarios:
    ax[0].plot(
        noch4r.timebound, 
        np.median(noch4r_lifetime.loc[dict(scenario=scenario)].__xarray_dataarray_variable__, axis=1), color=colors[scenario],
        ls = '--'
    );
    ax[0].plot(
        ch4r.timebound, 
        np.median(ch4r.lifetime.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax[0].get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax[0].set_xlim(2000, 2300)
ax[0].set_ylim(6, 11)
ax[0].set_ylabel('yr')
ax[0].axhline(0, lw=0.5, ls=':', color='k')
#ax[0].legend(hands, labs, frameon=False)
ax[0].set_title('(a) Methane lifetime');

for scenario in scenarios:
    ax[1].plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='CH4')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax[1].plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_ch4.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
#hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax[1].set_xlim(2000, 2300)
ax[1].set_xlim(2000, 2300)
ax[1].set_ylim(-1, 0.8)
ax[1].axhline(0, lw=0.5, ls=':', color='k')
ax[1].set_ylabel('W m$^{-2}$')
ax[1].set_title('(b) Methane radiative forcing');

for scenario in scenarios:
    ax[2].plot(
        noch4r.timebound, 
        np.median(noch4r.forcing.loc[dict(scenario=scenario, specie='Ozone')], axis=1), color=colors[scenario],
        ls='--',
    );
    ax[2].plot(
        ch4r.timebound, 
        np.median(ch4r.forcing_o3.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax[1].get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax[2].set_xlim(2000, 2300)
ax[2].set_ylim(-0.1, 0.6)
ax[2].set_ylabel('W m$^{-2}$')
ax[2].axhline(0, lw=0.5, ls=':', color='k')
ax[2].legend(hands, labs, frameon=False)
ax[2].set_title('(c) ozone radiative forcing');

fig.tight_layout()
pl.savefig('../plots/CH4lifetime-O3forcing_cal-1.4.0.png')
pl.savefig('../plots/CH4lifetime-O3forcing_cal-1.4.0.pdf')

In [ ]:
noch4r_lifetime.loc[dict(scenario=scenario)]

In [ ]:
fig, ax = pl.subplots(figsize=(9/2.54, 9/2.54))
for scenario in scenarios:
    ax.plot(
        noch4r.timebound, 
        np.median(noch4r_lifetime.loc[dict(scenario=scenario)].__xarray_dataarray_variable__, axis=1), color=colors[scenario],
        ls = '--'
    );
    ax.plot(
        ch4r.timebound, 
        np.median(ch4r.lifetime.loc[dict(scenario=scenario)], axis=1), color=colors[scenario],
        label=labels[scenario]
    );
hands, labs = ax.get_legend_handles_labels()
hands.append(Line2D([0], [0], color='k', lw=1.5))
hands.append(Line2D([0], [0], color='k', ls='--', lw=1.5))
labs.append('With methane removal')
labs.append('Unaltered scenario')
ax.set_xlim(2000, 2300)
ax.set_ylim(6, 11)
ax.set_ylabel('yr')
ax.axhline(0, lw=0.5, ls=':', color='k')
ax.legend(hands, labs, frameon=False)
ax.set_title('Ensemble median methane lifetime');
pl.savefig('../plots/CH4lifetime_cal-1.4.0.png')
pl.savefig('../plots/CH4lifetime_cal-1.4.0.pdf')

In [ ]:
noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')].shape

In [ ]:
def lighten_color(color, amount=0.5):
    """
    Lightens the given color by multiplying (1-luminosity) by the given amount.
    Input can be matplotlib color string, hex string, or RGB tuple.

    Examples:
    >> lighten_color('g', 0.3)
    >> lighten_color('#F034A3', 0.6)
    >> lighten_color((.3,.55,.1), 0.5)
    """
    import matplotlib.colors as mc
    import colorsys
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], 1 - amount * (1 - c[1]), c[2])

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):

    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            99,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.2),
        label='99%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            95,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.4),
        label='95%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            90,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.6),
        label='90%',
    );
    
    ax[iscen].fill_between(
        ch4r.timepoint, 
        np.percentile(
            (
                noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] -
                ch4r.emissions.loc[dict(scenario=scenario)]
            ),
            80,
            axis=1
        ),
        color=lighten_color(colors[scenario], 0.8),
        label='80%',
    );
    
    ax[iscen].plot(
        ch4r.timepoint, 
        np.median(noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)], axis=1), 
        color=colors[scenario],
        label='50%',
    );
    ax[iscen].set_xlim(2000, 2300)
    ax[iscen].set_ylim(0,1500)
    #ax.set_ylim(0, 900)
    if iscen==0:
        ax[iscen].set_ylabel('Tg CH$_4$ yr$^{-1}$')
    ax[iscen].legend(frameon=False)
    ax[iscen].set_title(f'({chr(iscen+97)}) Gross removal {labels[scenario]}');
fig.tight_layout()
pl.savefig('../plots/grossCH4removalpercentiles_cal-1.4.0.png')
pl.savefig('../plots/grossCH4removalpercentiles_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        ecs_tcr["TCR"][exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        ecs_tcr["TCR"][~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    if iscen==0:
        ax[iscen].set_ylabel('Transient climate response (°C)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(0.85, 3.05)
fig.tight_layout()
pl.savefig('../plots/cumCH4_TCR_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_TCR_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        ecs_tcr["ECS"][exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        ecs_tcr["ECS"][~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    if iscen==0:
        ax[iscen].set_ylabel('Equilibrium climate sensitivity (°C)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(1, 7.5)
fig.tight_layout()
pl.savefig('../plots/cumCH4_ECS_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_ECS_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        (
            noch4r.forcing.loc[
                dict(scenario=scenario, specie='Aerosol-radiation interactions', timebound=np.arange(2005, 2015))
            ].mean(dim='timebound')+noch4r.forcing.loc[
                dict(scenario=scenario, specie='Aerosol-cloud interactions', timebound=np.arange(2005, 2015))
            ].mean(dim='timebound')
        )[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        (
            noch4r.forcing.loc[
                dict(scenario=scenario, specie='Aerosol-radiation interactions', timebound=np.arange(2005, 2015))
            ].mean(dim='timebound')+noch4r.forcing.loc[
                dict(scenario=scenario, specie='Aerosol-cloud interactions', timebound=np.arange(2005, 2015))
            ].mean(dim='timebound')
        )[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    if iscen==0:
        ax[iscen].set_ylabel('2005-2014 aerosol forcing (W m$^{-2}$)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-2.7, 0.1)
fig.tight_layout()
pl.savefig('../plots/cumCH4_aerosolF_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_aerosolF_cal-1.4.0.pdf')

In [ ]:
zec = pd.read_csv('../results/zec_esm-bell_cal-1.4.0.csv', index_col=0)
zec

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        zec.zec50_1000[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        zec.zec50_1000[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    if iscen==0:
        ax[iscen].set_ylabel('ZEC50, °C (1000 PgC bell experiment)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-0.35, 0.55)
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')   
fig.tight_layout()
pl.savefig('../plots/cumCH4_zec50_1000_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_zec50_1000_cal-1.4.0.pdf')

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for iscen, scenario in enumerate(scenarios):
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[exceeds[scenario]]/1000, 
        zec.zec100_1000[exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0.4,
        alpha=0.8,
        edgecolor=colors[scenario],
        color="None",
        marker="o"
    )
    ax[iscen].scatter(
        -np.sum(
            ch4r.emissions.loc[
                dict(scenario=scenario)
            ] - noch4r.emissions.loc[
                dict(scenario=scenario, specie='CH4')
            ], axis=0
        )[~exceeds[scenario]]/1000, 
        zec.zec100_1000[~exceeds[scenario]],
        label=labels[scenario],
        s=8,
        lw=0,
        alpha=0.8,
        color=colors[scenario],
        marker="o"
    )
    ax[iscen].set_xlabel('Cumulative methane removal (PgCH$_4$)')
    if iscen==0:
        ax[iscen].set_ylabel('ZEC100 °C (1000 PgC bell experiment)')
    ax[iscen].set_xlim(-5, 130)
    ax[iscen].set_ylim(-0.55, 0.95)
    ax[iscen].set_title(f'({chr(iscen+97)}) {labels[scenario]}')
fig.tight_layout()
pl.savefig('../plots/cumCH4_zec100_1000_cal-1.4.0.png')
pl.savefig('../plots/cumCH4_zec100_1000_cal-1.4.0.pdf')

In [ ]:
ch4_obs = pd.read_csv('../data/constraints/IGCC_methane_concentrations_1750-2022.csv', index_col=0)
gmst_obs = pd.read_csv('../data/constraints/IGCC_GMST_1850-2022.csv', index_col=0)

In [ ]:
fig, ax = pl.subplots(1, 3, figsize=(18/2.54, 6/2.54))
for scenario in scenarios:
    ax[0].fill_between(
        np.arange(1750, 2023), 
        ch4r.concentration.sel(scenario=scenario, timebound=np.arange(1750, 2023)).min(dim='config'),
        ch4r.concentration.sel(scenario=scenario, timebound=np.arange(1750, 2023)).max(dim='config'),
        color=colors[scenario],
        alpha=0.5,
        lw=0
    );
    ax[0].plot(
        np.arange(1750, 2023), 
        ch4r.concentration.sel(scenario=scenario, timebound=np.arange(1750, 2023)).median(dim='config'),
        color=colors[scenario],
        lw=1,
        label=f'{labels[scenario]}',
    );
ax[0].plot(ch4_obs.index+0.5, ch4_obs, color='k', lw=1, label='observations')
ax[0].legend(frameon=False)
ax[0].set_xlim(1850, 2025);
ax[0].set_ylim(700, 2000);
ax[0].set_ylabel('ppb')
ax[0].set_title('(a) methane concentrations')

for scenario in scenarios:
    ax[1].plot(
        np.arange(1750, 2023), 
        ch4r.lifetime.sel(scenario=scenario, timebound=np.arange(1750, 2023)).median(dim='config'), color=colors[scenario],
    );
ax[1].set_xlim(1850, 2025);
ax[1].set_ylim(9, 11.25);
ax[1].set_ylabel('yr')
ax[1].set_title('(b) methane lifetime')

for scenario in scenarios:
    ax[2].fill_between(
        np.arange(1750, 2023), 
        (
            ch4r.temperature.sel(scenario=scenario, timebound=np.arange(1750, 2023))
          - np.average(noch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024), layer=0)], weights=temp_norm_20032022, axis=0)
          + 1.03
        ).quantile(0.05, dim='config'),
        (
            ch4r.temperature.sel(scenario=scenario, timebound=np.arange(1750, 2023))
          - np.average(noch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024), layer=0)], weights=temp_norm_20032022, axis=0)
          + 1.03
        ).quantile(0.95, dim='config'),
        color=colors[scenario],
        alpha=0.5,
        lw=0
    );
    ax[2].plot(
        np.arange(1750, 2023), 
        (
            noch4r.temperature.sel(scenario=scenario, timebound=np.arange(1750, 2023), layer=0) -
            np.average(noch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024), layer=0)], weights=temp_norm_20032022, axis=0)
            + 1.03
        ).median(dim='config'),
        color=colors[scenario],
        lw=1,
        label=f'FaIR {labels[scenario]}',
    );
ax[2].plot(gmst_obs.index+0.5, gmst_obs, color='k', lw=1)
ax[2].set_xlim(1850, 2025);
ax[2].set_ylim(-0.3, 1.5);
ax[2].set_ylabel('°C relative to 1850-1900')
ax[2].set_title('(c) surface temperature')

fig.tight_layout()
pl.savefig('../plots/historical_cal-1.4.0.png')
pl.savefig('../plots/historical_cal-1.4.0.pdf')

In [ ]:
pl.plot(1.03 + ch4r.temperature[:, 0, :] - np.average(ch4r.temperature[253:274, 0, :].data, weights=temp_norm_20032022, axis=0));

In [ ]:
np.average(ch4r.temperature[253:274, 0, :].data, weights=temp_norm_20032022, axis=0)#.mean(axis=0)

In [ ]:
np.average(noch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024), layer=0)], weights=temp_norm_20032022, axis=0)

## Calculate effective cumulative removal

[Abernethy et al. 2021](https://royalsocietypublishing.org/doi/10.1098/rsta.2021.0104)

$E(t) = \int_0^t R(t') e^{-t'/\tau} dt'$

In [ ]:
scenario='ssp119'

In [ ]:
# R(t')
noch4r.emissions.loc[dict(scenario=scenario, specie='CH4')] - ch4r.emissions.loc[dict(scenario=scenario)]

In [ ]:
# tau
ch4r.lifetime.loc[dict(scenario=scenario)]

In [ ]:
# first R(t') where we see any action. This is 2024 to 2025, so could be the cumulative R at 2025.01.01
noch4r.emissions.loc[dict(scenario=scenario, specie='CH4', timepoint=2024.5)] - ch4r.emissions.loc[dict(scenario=scenario, timepoint=2024.5)]

In [ ]:
ch4r.lifetime.loc[dict(scenario=scenario, timebound=2025)]

In [ ]:
# first R(t') where we see any action. This is 2024 to 2025, so could be the cumulative R at 2025.01.01
(
    noch4r.emissions.loc[dict(scenario=scenario, specie='CH4', timepoint=2024.5)] - 
    ch4r.emissions.loc[dict(scenario=scenario, timepoint=2024.5)]
) / np.exp(-1 / ch4r.lifetime.loc[dict(scenario=scenario, timebound=2025)])

In [ ]:
# do a numerical integration
equivalent_marginal_removal = np.zeros((278, 841))
for itb, timebound in enumerate(range(2023, 2301)):
    equivalent_marginal_removal[itb, :] = (
        noch4r.emissions.loc[dict(scenario=scenario, specie='CH4', timepoint=timebound-0.5)] - 
        ch4r.emissions.loc[dict(scenario=scenario, timepoint=timebound-0.5)]
    ) / np.exp(-1 / ch4r.lifetime.loc[dict(scenario=scenario, timebound=timebound)])

In [ ]:
equivalent_cumulative_removal = np.nancumsum(equivalent_marginal_removal, axis=0)

In [ ]:
pl.plot(equivalent_cumulative_removal);

In [ ]:
delta_T = (
    noch4r.temperature.loc[dict(scenario=scenario, layer=0)] -
    np.average(noch4r.temperature.loc[dict(scenario=scenario, layer=0, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    + 1.03
) - (
    ch4r.temperature.loc[dict(scenario=scenario)] -
    np.average(ch4r.temperature.loc[dict(scenario=scenario, timebound=np.arange(2003,2024))], weights=temp_norm_20032022, axis=0).mean()
    + 1.03
)[273:, :]

In [ ]:
pl.plot(equivalent_cumulative_removal[:10, :], delta_T[:10, :]);